In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [48]:
data_set=pd.read_csv('train.csv')

In [49]:
test_set=pd.read_csv('test.csv')

In [50]:
labelencoder_X=LabelEncoder()

In [128]:
def data_prep(train,test):
    all_data=pd.concat([train.drop('Survived',1),test])
    #Feature Title, Master, miss and other titles give us the indication that the person was young, other titles looks that the person was relativly old
    all_data['Title']=all_data['Name'].map(lambda complete_name: complete_name.split(',')[1].split('.')[0].strip())
    titles=set(all_data['Title'])
    temp_data=all_data[:len(train)]
    titles_mapping=dict(zip(titles,range(0,len(titles)+1)))
    temp_data['Numeric_title']=temp_data.apply(lambda passanger:titles_mapping[passanger.Title],axis=1).astype(int)
    all_data['Numeric_title']=all_data.apply(lambda passanger:titles_mapping[passanger.Title],axis=1).astype(int)
    #Fill unkonw embarcation with the mode of the embarked zone of the train data
    all_data.loc[all_data.Embarked.isnull(),'Embarked']=train.Embarked.mode()[0]
    #check why next line doesnt return int, instead return the letter
    #all_data['Embarket']=all_data['Embarked'].map({'C': 0, 'S': 1, 'Q': 2}).astype(int)
    all_data['Embarked']=labelencoder_X.fit_transform(all_data['Embarked'])
    embarket_place=sorted(all_data['Embarked'].unique(), key=lambda x: x)
    #Now let's fill missing age with the average of each title class but only using the train
    average_age_title={}
    for i in range(0,len(titles_mapping)):
        average_age_title[i]=np.median(train.loc[temp_data.Numeric_title==i,('Age')].dropna())
    all_data.loc[all_data.Age.isnull(),('Age')]=all_data[all_data.Age.isnull()].apply(lambda element: average_age_title.get(element.Numeric_title),axis=1)
    #From histogram people up to 10 have better prob to survive, for people with more than 63 is the opposite
    all_data['Young']=all_data.apply(lambda element: int(element.Age<=10),axis=1)
    all_data['Late_adulthood']=all_data.apply(lambda element: int(element.Age>=63),axis=1)
    #family size, non_including his/her-self
    all_data['Family_size']=all_data['SibSp']+all_data['Parch']
    #If the person is not-alone has better chances to survive
    all_data['Alone']=all_data.apply(lambda element: int((element.SibSp+element.Parch)==0),axis=1)
    #young+alone seem like has less probablity to live, but maybe is not so important since in the train only 1 is young and alone
    all_data['Young_alone']=all_data.apply(lambda element: (element.Young==1)and(element.Alone==1),axis=1).astype(int)
    #Also bigger families lower prob to survive :(
    all_data['Big_family']=all_data.apply(lambda element: (element.SibSp>2)or(element.Parch>3),axis=1).astype(int)
    #Obviuslly sex has a big impact, female has better prob to survive, there are sklearn ways to do this but pandas is really nice
    all_data['Sex']=all_data['Sex'].map({'male': 0, 'female': 1}).astype(int)
    #There is some correlation between fare and embarket place so let's use the mean/median (need to pay attention to the skwekness) for each embarket place
    # however let;s do this in future versions
    
    average_fare_class=np.zeros([3,3])
    #there are 3 classes so let's use it
    for i in range(1,4):
        for j in range(1,4):
            average_fare_class[i-1][j-1]=(np.nanmedian((train.Pclass==i)&(train.Embarked==j)))
    all_data.loc[all_data.Fare.isnull(),('Fare')]=all_data[all_data.Fare.isnull()].apply(lambda element: average_fare_class[element.Pclass-1][element.Embarked-1],axis=1)
        
    return all_data[:len(train)],all_data[len(train):]
    
    
    
    

In [129]:
train,test=data_prep(data_set,test_set)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [130]:
len(train)

891

In [131]:
len(test)

418

In [132]:
X_train=train.drop(['Ticket','Cabin','Name','Title'],axis=1)

In [133]:
survived_train=data_set['Survived']

In [134]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold


In [135]:
kf=KFold(n_splits=3)
parameters={'n_estimators':[10,50,100,150,200],'max_depth':[3,4,5,6,7,8,9,10]}

In [136]:
clf=RandomForestClassifier()

In [137]:
from sklearn.model_selection import GridSearchCV

In [138]:
grid_search=GridSearchCV(clf,param_grid=parameters,cv=kf)

In [141]:
#X_train['Embarked']=labelencoder_X.fit_transform(X_train['Embarked'])
#X_train['Sex']=labelencoder_X.fit_transform(X_train['Sex'])


In [142]:
grid_search.fit(X_train,survived_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100, 150, 200], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [143]:
#X_train.isnull().values.any()

In [144]:
#nan_rows = X_train[X_train.isnull().any(1)]

In [145]:
grid_search.best_score_

0.83277216610549942

In [146]:
grid_search.best_params_

{'max_depth': 5, 'n_estimators': 50}

In [147]:
clf_new=RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],max_depth=grid_search.best_params_['max_depth'])

In [148]:
clf_new=clf_new.fit(X_train,survived_train)

In [149]:
y_test_predicted=clf_new.predict(X_train)

In [150]:
y_test=test.drop(['Ticket','Cabin','Name','Title'],axis=1)

In [151]:
#y_test['Embarked']=labelencoder_X.fit_transform(y_test['Embarked'])

In [152]:
survived_prediction=clf_new.predict(y_test)

In [153]:
submission=pd.DataFrame({"PassengerID":y_test['PassengerId'],"Survived":survived_prediction})

In [154]:
submission.to_csv('random_forrest_submission_with_gridsearch_and_more_features_with_fare.csv',index=False)